# Web Scraping con Selenium

##### Ejercicio en grupos


##### En parejas, vamos a buscar las cuatro primeras películas en cartelera. De ellas vamos a extraer la siguiente información:
- ##### Fecha de estreno
- ##### URL
- ##### Datos principales, como hemos hecho al principio
- ##### Nota media
- ##### Cantidad de votos
- ##### Críticas profesionales buenas, regulares y malas
- ##### Cinco primeras críticas

In [1]:
#%pip install selenium

In [2]:
#%pip install webdriver-manager

Importación de las librerías

In [3]:
#Para la manipulación de datos
import pandas as pd

#Servicio y driver de Chrome de Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#Las opciones que vamos a tener para buscar elementos
from selenium.webdriver.common.by import By

#Para esperar a que carguen las páginas
import time


Creamos el driver para controlar el navegador

In [4]:
service=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

RECUERDA:

##### Beginner Selenium Cheatsheet:
Sacar un elemento:
- element = driver.find_element(by, value)

Sacar varios elementos:
- element = driver.find_elements(by, value)

Sacar atributos:
- attribute = element.--el atributo--
- attribute = element.get_attribute(--el atributo--)

Hacer click:
- element.click()

Teclear:
- element.send_keys()

Gestión de pestañas:
- driver.switch_to.window(driver.window_handles[-1])
- driver.get(url)
- driver.close()

Accedemos a la página principal

In [5]:
driver.get("https://www.filmaffinity.com/")

Aceptamos el pop-up de ser necesario

In [6]:
#Seleccionamos el botón
accept = driver.find_element(by=By.CLASS_NAME, value='css-v43ltw')

#Hacemos click
accept.click()

Hacemos una función que devuelva en un diccionario todos los datos de las películas, salvo la fecha de estreno y la url

Parámetros: url y fecha de estreno
Salida: Diccionario con todos los datos

In [7]:
def get_datos_pelicula(driver, url, estreno):

    #Creamos el diccionario
    movie_dict = dict()

    #Añadimos al diccionario los parámetros que nos vienen dados
    movie_dict['fecha_estreno'] = estreno
    movie_dict['url'] = url

    #Accedemos a la url
    driver.get(url)
    time.sleep(2)

    #Sacamos los datos principales
    principales = driver.find_element(by=By.CLASS_NAME, value='movie-info')

    dts = principales.find_elements(by=By.TAG_NAME, value='dt')
    dds = principales.find_elements(by=By.TAG_NAME, value='dd')

    i = 0
    while i < len(dts):
        movie_dict[dts[i].text] = dds[i].text
        i += 1

    #Sacamos la nota media
    nota_media = driver.find_elements(by=By.ID, value="movie-rat-avg")
    if len(nota_media) > 0:
        movie_dict['nota_media'] = nota_media[0].text

    #Sacamos la cantidad de votos
    cantidad_votos = driver.find_elements(by=By.CSS_SELECTOR, value="#movie-count-rat span")
    if len(cantidad_votos) > 0:
        movie_dict['cantidad_votos'] = cantidad_votos[0].text

    #Críticas profesionales
    n_criticas = driver.find_elements(by=By.CSS_SELECTOR, value='#right-column > a > div > div.body > div > div.legend-wrapper .leg')

    if len(n_criticas) > 0:
        #Criticas profesionales positivas
        positivas = n_criticas[0].text
        movie_dict['positivas'] = positivas

        #Criticas profesionales regulares
        regulares = n_criticas[1].text
        movie_dict['regulares'] = regulares

        #Criticas profesionales negativas
        negativas = n_criticas[2].text
        movie_dict['negativas'] = negativas

    #Cinco primeras criticas
    criticas = driver.find_elements(by=By.CSS_SELECTOR, value="ul#pro-reviews li")

    i = 0

    while i < 5 and i < len(criticas):
        critica = criticas[i].find_element(by=By.CSS_SELECTOR, value='div div').text
        movie_dict['critica_'+str(i)] = critica

        i += 1


    return movie_dict


Probamos la función que hemos hecho. Aquí tienes un enlace de prueba: https://www.filmaffinity.com/es/film599984.html

In [8]:
prueba = get_datos_pelicula(driver, "https://www.filmaffinity.com/es/film618375.html", "fecha")
prueba

{'fecha_estreno': 'fecha',
 'url': 'https://www.filmaffinity.com/es/film618375.html',
 'Título original': 'Oblivion',
 'Año': '2013',
 'Duración': '126 min.',
 'País': ' Estados Unidos',
 'Dirección': 'Joseph Kosinski',
 'Guion': 'Joseph Kosinski, Michael Arndt, Karl Gajdusek. Cómic: Joseph Kosinski, Arvid Nelson',
 'Reparto': 'Tom Cruise\nAndrea Riseborough\nOlga Kurylenko\nMorgan Freeman\nNikolaj Coster-Waldau\nZoe Bell',
 'Música': 'Anthony Gonzalez, M83, Joseph Trapanese',
 'Fotografía': 'Claudio Miranda',
 'Compañías': 'Universal Pictures, Chernin Entertainment, Relativity Studios, Monolith Pictures, Radical Studios',
 'Género': 'Ciencia ficción. Intriga | Futuro postapocalíptico. Distopía. Cómic',
 'Sinopsis': 'Año 2073. Hace más de 60 años la Tierra fue atacada; se ganó la guerra, pero la mitad del planeta quedó destruido, y todos los seres humanos fueron evacuados. Jack Harper (Tom Cruise), un antiguo marine, es uno de los últimos hombres que la habitan. Es un ingeniero de Dron

Entramos en el link de las películas en cartelera

In [9]:
driver.find_elements(By.PARTIAL_LINK_TEXT, value="Películas en cartelera")[0].click()

Sacamos todas las películas y llamamos a la función con cada película

In [10]:
#Creamos una lista vacia
links = []

#Sacamos el elemento raíz
filas = driver.find_elements(by=By.CSS_SELECTOR, value='#wrapper-cat div.row')

for fila in filas:

    peliculas = fila.find_elements(by=By.CLASS_NAME, value='movie-poster')

    for pelicula in peliculas:
        url = pelicula.find_element(By.TAG_NAME, value='a').get_attribute('href')
        estreno = pelicula.find_element(By.CSS_SELECTOR, value='a div.release-text').text.replace('\n', ' ')

        link_pelicula = {
            'url': url,
            'estreno': estreno
        }

        links.append(link_pelicula)


Ahora usamos los links para llamar a la funcion y sacar los datos

In [12]:
#Reservamos la variable para el dataframe
df = None

for link in links:
    datos_pelicula = get_datos_pelicula(driver, link['url'], link['estreno'])

    #Si no está creado lo creamos
    if df is None:
        df = pd.DataFrame(columns=datos_pelicula.keys())

    #Metemos los datos de la película
    #df = df.append(datos_pelicula, ignore_index=True)
    df = pd.concat([df, pd.DataFrame(datos_pelicula, index = [0])], ignore_index=True)
    print(f"Añadida {datos_pelicula['Título original']}")

Añadida Wonkaaka
Añadida Robot Dreams
Añadida Anatomie d'une chute
Añadida The Piper
Añadida Maestro
Añadida Momonsters: La película
Añadida La terra delle donne
Añadida Chicken Run: Dawn of the Nugget
Añadida Brother
Añadida NCT NATION: To The World in Cinemas
Añadida Ocho apellidos marroquísaka
Añadida La Navidad en sus manos
Añadida Silent Night
Añadida Digimon Adventure 02: The Beginning
Añadida Chlopiaka
Añadida She Came to Me
Añadida Sobre todo de noche
Añadida Una noche con Adelaaka
Añadida Callas - Paris, 1958aka
Añadida Los delincuentes
Añadida Cristina García Rodero: La mirada oculta
Añadida Folk og røvere i Kardemomme byaka
Añadida La mesita del comedor
Añadida Quest
Añadida Napoleon
Añadida Wish
Añadida Teresa
Añadida El amor de Andrea
Añadida Scrapper
Añadida The Hunger Games: The Ballad of Songbirds and Snakes
Añadida Thanksgiving
Añadida La ermita
Añadida Que nadie duerma
Añadida The Old Oak
Añadida The Marvels
Añadida El favor
Añadida Un amor
Añadida Eiga Doraemon: Nobi

In [13]:
df

,fecha_estreno,url,Título original,,Año,Duración,País,Dirección,Guion,Reparto,...,nota_media,cantidad_votos,positivas,regulares,negativas,critica_0,critica_1,critica_2,critica_3,critica_4
0,6 dic.,https://www.filmaffinity.com/es/film430111.html,Wonkaaka,,2023,115 min.,Reino Unido,Paul King,"Simon Farnaby, Paul King. Personaje: Roald Dahl",Timothée Chalamet\nCalah Lane\nOlivia Colman\n...,...,"6,8",1.364,34,7,1,"""Es la merienda perfecta (...) Sin llegar a la...","""Deliciosa, divertida y exagerada fábula con u...","""Es el mejor “mundo de fantasía” hasta la fech...","""La maníaca atención al detalle de la direcció...","""Brilla tanto el diseño de producción como las..."
1,6 dic.,https://www.filmaffinity.com/es/film399287.html,Robot Dreams,NaN,2023,102 min.,España,Pablo Berger,Pablo Berger. Novela gráfica: Sara Varon,Animación,...,"7,3",830,21,1,1,"""Muda, tierna, sabia, adulta e infantil. Dicho...","""Una propuesta tan clásica como revolucionaria...","""Asombrosa (...) conmovedora delicia, que, des...","""Una maravilla para alumbrar a la infancia y d...","""Una película tierna, llena de imaginación. Un..."
2,6 dic.,https://www.filmaffinity.com/es/film108208.html,Anatomie d'une chute,NaN,2023,150 min.,Francia,Justine Triet,"Arthur Harari, Justine Triet",Sandra Hüller\nSamuel Theis\nSwann Arlaud\nJeh...,...,"7,5",2.001,36,4,0,"""La directora disecciona de forma bastante aso...","""Una pieza estudiadísima de cine procedimental...","""Gracias a una sucesión de interrogatorios y d...","""Hay mucha eficacia narrativa, mucha anatomía,...","""Retorcida, ambigua y aceptable intriga. Sigo ..."
3,6 dic.,https://www.filmaffinity.com/es/film831239.html,The Piper,NaN,2023,95 min.,Estados Unidos,Erlingur Thoroddsen,Erlingur Thoroddsen,Charlotte Hope\nJulian Sands\nKate Nichols\nAl...,...,"4,9",50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6 dic.,https://www.filmaffinity.com/es/film608753.html,Maestro,NaN,2023,129 min.,Estados Unidos,Bradley Cooper,"Josh Singer, Bradley Cooper. Biografía sobre: ...",Bradley Cooper\nCarey Mulligan\nMatt Bomer\nMa...,...,"6,3",97,28,8,5,"""Cooper luce irreconocible. Y su actuación, ju...","""Magistral biopic (...) un film que consolida ...","""Se esfuerza por concentrar nuestra atención (...","""Bradley Cooper convierte el 'biopic' del músi...","""Hay tantísimo esfuerzo, tantas ganas de proba..."
5,6 dic.,https://www.filmaffinity.com/es/film761600.html,Momonsters: La película,NaN,2023,70 min.,España,"Javier Martinez, Christian Garnez",NaN,Animación,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6 dic.,https://www.filmaffinity.com/es/film356802.html,La terra delle donne,NaN,2023,104 min.,Italia,Marisa Vallone,"Paola Sini, Marisa Vallone",Paola Sini\nValentina Lodovini\nJan Bijvoet\nS...,...,NaN,NaN,NaN,NaN,NaN,"""Película contradictoria y sugerente a la vez....",NaN,NaN,NaN,NaN
7,6 dic.,https://www.filmaffinity.com/es/film186480.html,Chicken Run: Dawn of the Nugget,NaN,2023,101 min.,Reino Unido,Sam Fell,"Karey Kirkpatrick, John O'Farrell, Rachel Tunn...",Thandiwe Newton\nZachary Levi\nBella Ramsey\nI...,...,NaN,NaN,5,5,3,"""Aunque la aventura mantiene nuestra atención,...","""Un delicioso regreso a las raíces de stop-mot...","""Roza la fórmula y da la sensación de haber si...","""Una emocionante secuela llena de humor que se...","""Hay muchísima innovación en lo visual pero ap..."
8,6 dic.,https://www.filmaffinity.com/es/film640518.html,Brother,NaN,2021,78 min.,Francia,Arnaud Fournier Montgieux,Arnaud Fournier Montgieux,Documental,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6 dic.,https://www.filmaffinity.com/es/film837907.html,NCT NATION: To The World in Cinemas,NaN,2023,103 min.,Corea del Sur,Yoon Dong Oh,NaN,NCT 127\nNCT Dream\nDoyoung\nHaechan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
